# Simulation

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Generate random uniform numbers

In [2]:
n = 1000000

In [3]:
runs = 100000

In [4]:
from numpy.random import default_rng
rng = default_rng()

## Generate GM indifference numbers

In [5]:
from sympy.solvers import solve
from sympy import Symbol, binomial, summation, N

In [6]:
x = Symbol('x', positive=True)
j = Symbol('j')

In [7]:
def eqin(i):
    return summation((binomial(i, j) * (x ** (i - j)) * ((1 - x) ** j)) / j, (j, 1, i))

In [8]:
def solvi(i):
    sol = solve([x**(i - 1) -  eqin(i - 1), x < 1], x)
    return N(sol.rhs)

In [9]:
%%time

gmd100 = pd.DataFrame([solvi(i) for i in range(2, 100 + 1)], columns=['ki'], index=range(2, 100 + 1))

CPU times: user 2min 17s, sys: 167 ms, total: 2min 17s
Wall time: 2min 18s


In [10]:
gmd100.index.name = 'i'
gmd100.loc[1] = 0.
gmd100.sort_index(inplace=True)

In [11]:
gmd100

,ki
i,
1,0.000000
2,0.500000000000000
3,0.689897948556636
4,0.775845067578928
5,0.824589583005756
...,...
96,0.991584247313878
97,0.991671387317630
98,0.991756741258583


In [12]:
def solv2ndordi(i):
    return 1 / (1 + 0.80435226286 / (i - 1) + 0.183199 / (i - 1) ** 2)

In [13]:
%%time

gmdn = pd.DataFrame([solv2ndordi(i) for i in range(2, n + 1)], columns=['ki'], index=range(2, n + 1))

CPU times: user 535 ms, sys: 22.6 ms, total: 558 ms
Wall time: 557 ms


In [14]:
gmdn.index.name = 'i'
gmdn.loc[1] = 0.
gmdn.sort_index(inplace=True)

In [15]:
gmdn.loc[:100]

,ki
i,
1,0.000000
2,0.503132
3,0.690619
4,0.776113
5,0.824716
...,...
96,0.991584
97,0.991671
98,0.991757


In [28]:
ksgm = np.flip(gmdn.values.T[0])

## Generate MC cutoff numbers

In [22]:
ksmc = np.array([((1 - 1 / n) + np.log((n - r) / n) / n) for r in range(1, n)] + [0])

## Run both choices

In [35]:
def compare(xsr, ks1, ks2, ctr):
    rmax = np.argmax(xsr)
    diff1 = np.sign(xsr - ks1)
    rpick1 = np.argmax(diff1)
    tp1, fp1, fn1 = [0, 0, 0]
    diff2 = np.sign(xsr - ks2)
    rpick2 = np.argmax(diff2)
    tp2, fp2, fn2 = [0, 0, 0]
    if rpick1 == rmax:
        tp1 = 1   
    else:
        if rpick1 < rmax:
            fp1 = 1
        else:
            fn1 = 1
    if rpick2 == rmax:
        tp2 = 1   
    else:
        if rpick2 < rmax:
            fp2 = 1
        else:
            fn2 = 1
    if ctr % 1000 == 0:
        print(ctr)
    return [rmax + 1, rpick1 + 1, rpick2 + 1, tp1, fp1, fn1, tp2, fp2, fn2]

In [38]:
%%time

results = pd.DataFrame([compare(rng.random(size=n), ksgm, ksmc, run) for run in range(runs)],
                      columns=['R(Max)', 'R(Choice)-GM', 'R(Choice)-MC',
                               'TP-GM', 'FP-GM', 'FN-GM', 'TP-MC', 'FP-MC', 'FN-MC'])

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
CPU times: user 18min 47s, sys: 3min 36s, total: 22min 23s
Wall time: 22min 24s


In [39]:
results.mean()

R(Max)          498436.28951
R(Choice)-GM    553636.06085
R(Choice)-MC    537557.75820
TP-GM                0.53869
FP-GM                0.26174
FN-GM                0.19957
TP-MC                0.56211
FP-MC                0.25376
FN-MC                0.18413
dtype: float64